In [ ]:
!pip install Historic-Crypto
from Historic_Crypto import HistoricalData
from Historic_Crypto import LiveCryptoData
from Historic_Crypto import Cryptocurrencies
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np
import plotly.graph_objects as go
import gzip
import io

from google.colab import drive
drive.mount('/content/drive')

In [2]:
# Fetch OHLCV data
def fetch_data(ticker='BTC-USD', granularity=60, start_date='2017-07-22-00-00', end_date=None, verbose=False):
  df = HistoricalData(ticker, granularity, start_date, end_date, verbose).retrieve_data()
  return df


def check_raw_data(df, freq='D'):
  print('---------------------------------------------')
  print(f'Range of Data: {df.index.min()} - {df.index.max()}')
  print('---------------------------------------------\n')
  print('---------------------------------------------')
  if df.isna().any().any():
    print('There are missing values in the raw data')
  else:
    print('The raw data is complete')
  print('---------------------------------------------\n')

  missing_days = pd.date_range(start=df.index.floor('d').min(), end=df.index.floor('d').max(), freq=freq).difference(df.index.floor('d'))
  print('Missing days: ', missing_days.size)
  print(missing_days)
  print('---------------------------------------------\n')
  print('---------------------------------------------')
  print('Shape:', df.shape)
  print('---------------------------------------------\n')


def sort_unique(df):
  print('---------------------------------------------')
  if df.index.is_monotonic_increasing == False:
    df.sort_index(inplace=True)
  print('Dataset is monotonic increasing')
  print('---------------------------------------------\n')
  print('---------------------------------------------')
  duplicate_indices = df.index.duplicated(keep=False)
  duplicated_df = df[duplicate_indices]
  if duplicated_df.shape[0] == 0:
    print('The dataset has no duplicate values')
    print('---------------------------------------------')
    df_ = df
  else:
    df_ = df[~df.index.duplicated(keep='first')]
    print(' Duplicates droped')
    print('---------------------------------------------')
    print('---------------------------------------------')
    print('New Shape:', df_.shape)
    print('---------------------------------------------\n')
  return df_


def save_data(df, name):
  # on Drive
  folder_path = '/content/drive/MyDrive/Colab Notebooks/MA/Daten/'
  file_path = folder_path + name + '.csv'
  df.to_csv(file_path)
  # Bestätigungsnachricht ausgeben
  print(f"Die Datei wurde erfolgreich in {file_path} gespeichert.")

  # Environment -> local
  #df.to_csv('df_16_17.csv') 


# Analyze trading volume of each day
def vol_per_day(df):
  df_by_ = df.groupby(df.index.date)
  volume_by_ = df_by_['volume'].sum()
  plt.hist(volume_by_, bins=50)
  plt.xlabel('Trading Volume')
  plt.ylabel('Days')
  plt.show()

